In [1]:
# Check GPU
!nvidia-smi

Fri May 31 13:07:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:00:05.0 Off |                    0 |
| N/A   31C    P0              41W / 300W |      0MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
# Import packages
from IPython.display import display, Image
from IPython import display
import csv
import os
import json
from shapely.geometry import Polygon
import pandas as pd

In [3]:
# Import roboflow
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="YgqaFYQYMXIDYOCYoY2O") # Fill in your person api_key
project = rf.workspace("footage").project("seafloor-segmentation") # Fill in your workspace and project name
model = project.version("1").model

display.clear_output()
print("Successfully extracted data from roboflow")

Successfully extracted data from roboflow


In [11]:
def create_output_dir(base_dir):
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
        return base_dir
    else:
        counter = 2
        while True:
            new_dir = f"{base_dir}{counter}"
            if not os.path.exists(new_dir):
                os.makedirs(new_dir)
                return new_dir
            counter += 1

base_output_dir = "/cache/album/cache/kso-benjamin/bucket/kso/notebooks/test/predictions/seg-pred" # Define base dir
output_dir = create_output_dir(base_output_dir)

input_directory = "/cache/album/cache/kso-benjamin/bucket/kso/notebooks/test/Seafloor_footage-1/Raw/" # Define dir with raw images

for filename in os.listdir(input_directory): # Iterate over the files
    filepath = os.path.join(input_directory, filename) # Get the full path 
    
    if filepath.lower().endswith(('.png', '.jpg', '.jpeg')): # Check format

        prediction = model.predict(filepath, confidence=40) # Perform prediction on the image
        
        prediction_data = prediction.json() # Get the JSON results
        
        csv_filename = os.path.splitext(filename)[0] + "_prediction.csv" # Write to csv
        csv_filepath = os.path.join(output_dir, csv_filename)
        
        with open(csv_filepath, mode='w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            
            header = ["x", "y", "width", "height", "confidence", "class", "class_id", "detection_id", "image_path", "prediction_type", "points", "area"] # Write the header row
            csv_writer.writerow(header)

            image_area = 0
            quadrant_area = 0
            rows = []
            for idx, pred in enumerate(prediction_data['predictions']):  # Write each prediction as a row
                mask_points = pred.get('points', [])
                
                if mask_points:
                    try:
                        mask_points = [(float(point['x']), float(point['y'])) for point in mask_points]
                        polygon = Polygon(mask_points)
                        area = polygon.area
                    except ValueError as e:
                        print(f"Error converting points to float for {filename}: {e}")
                        area = 0
                else:
                    area = 0
                
                if pred.get('class', '') == 'quadrant':
                    x = float(pred.get('x', 0))
                    y = float(pred.get('y', 0))
                    width = float(pred.get('width', 0))
                    height = float(pred.get('height', 0))
                    quadrant_area = width * height
                
                row = {
                    "x": pred.get('x', ''),
                    "y": pred.get('y', ''),
                    "width": pred.get('width', ''),
                    "height": pred.get('height', ''),
                    "confidence": pred.get('confidence', ''),
                    "class": pred.get('class', ''),
                    "class_id": pred.get('class_id', ''),
                    "detection_id": pred.get('detection_id', ''),
                    "image_path": pred.get('image_path', ''),
                    "prediction_type": pred.get('prediction_type', ''),
                    "points": json.dumps(mask_points), # Convert mask points to JSON string
                    "area": area,
                }
                rows.append(row)
            
            for row in rows:
                csv_writer.writerow([
                    row["x"],
                    row["y"],
                    row["width"],
                    row["height"],
                    row["confidence"],
                    row["class"],
                    row["class_id"],
                    row["detection_id"],
                    row["image_path"],
                    row["prediction_type"],
                    row["points"],
                    row["area"]
                ])
        
        labeled_prediction = model.predict(filepath, confidence=30) # Perform prediction with labels and save the result
        
        output_image_path = os.path.join(output_dir, os.path.splitext(filename)[0] + "_prediction.jpg") # Construct the output file path for labeled prediction image
        
        labeled_prediction.save(output_image_path) # Save the labeled prediction result to an image file
        
        display.clear_output()

def calculate_area_percentages(output_dir):
    results = {}
    all_classes = set()

    for filename in os.listdir(output_dir):
        if filename.endswith('_prediction.csv'):
            csv_filepath = os.path.join(output_dir, filename)
            class_area_percentages = {}
            total_quadrant_area = 0
            total_image_area = 0
            non_quadrant_detections = False 
            
            with open(csv_filepath, mode='r') as csv_file:
                csv_reader = csv.DictReader(csv_file)
                for row in csv_reader:
                    class_name = row['class']
                    
                    if class_name == "quadrant":
                        width = float(row['width'])
                        height = float(row['height'])
                        total_quadrant_area = width * height
                        class_area_percentages[class_name] = total_quadrant_area
                        all_classes.add(class_name)
                        continue

                    area = float(row.get('area', 0))
                    
                    if area > 0:
                        non_quadrant_detections = True
                    
                    if class_name in class_area_percentages:
                        class_area_percentages[class_name] += area
                    else:
                        class_area_percentages[class_name] = area
                    all_classes.add(class_name)
            
            image_name = filename.replace('_prediction.csv', '')
            results[image_name] = {}

            if total_quadrant_area == 0:
                image_path = os.path.join(input_directory, image_name + '.jpg')
                with Image.open(image_path) as img:
                    total_image_area = img.width * img.height
                    
            if non_quadrant_detections:
                for class_name, total_area in class_area_percentages.items():
                    if class_name == "quadrant":
                        results[image_name][class_name] = 100.0
                    elif total_quadrant_area > 0:
                        area_percentage = (total_area / total_quadrant_area) * 100
                        results[image_name][class_name] = area_percentage
                    else:
                        area_percentage = (total_area / total_image_area) * 100
                        results[image_name][class_name] = area_percentage
            else:
                results[image_name] = {'No detections besides the quadrant': 'Na'}

    return results, all_classes

def write_results_to_csv(results, all_classes, output_filepath):
    formatted_results = {}
    for image_name, class_areas in results.items():
        formatted_results[image_name] = {class_name: class_areas.get(class_name, 'Na') for class_name in all_classes}
    
    df = pd.DataFrame.from_dict(formatted_results, orient='index', columns=sorted(all_classes))
    df.index.name = 'Image Name'
    df.to_csv(output_filepath, na_rep='Na')

results, all_classes = calculate_area_percentages(output_dir)

results_csv_filepath = os.path.join(output_dir, 'results.csv')
write_results_to_csv(results, all_classes, results_csv_filepath)

print(f'Results have been written to {results_csv_filepath}')


Results have been written to /cache/album/cache/kso-benjamin/bucket/kso/notebooks/test/predictions/seg-pred/results.csv
